In [3]:
%load_ext autoreload
%autoreload 2

import numpy as np
from model.investment_planner import  get_goals_strategies, calculateValuesForLastPeriod, calculateBelmanForT 
from model.propabilities import calculateTransitionPropabilitiesForAllPorfolios, calculateTransitionPropabilitiesForGoals, __calculateWtc
from model.grid import generateGrid
from scipy.stats import norm

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [91]:

portfolios = np.array([[0.0526, 0.0374], [0.07059443, 0.103057  ], [0.0886, 0.1954]])
W0 = np.array([90,95,100,105])
W1 = np.array([100,103,105,110])
infusions = 2
cost = 3
h = 1

values = [0,0,100,100]

#propabilities = calculateTransitionPropabilitiesForAllPorfolios(portfolios, W0, W1, infusions, cost, h)
propabilities = np.array([[[0.451, 0.334, 0.197, 0.018],
                                    [0.271, 0.27,  0.258, 0.2  ],
                                    [0.255, 0.256, 0.253, 0.237]],

                                    [[0.14,  0.307, 0.366, 0.187],
                                    [0.221, 0.254, 0.266, 0.259],
                                    [0.24,  0.251, 0.255, 0.254]],

                                    [[0.017, 0.105, 0.244, 0.634],
                                    [0.177, 0.234, 0.268, 0.322],
                                    [0.227, 0.246, 0.256, 0.271]],

                                    [[0.001, 0.017, 0.075, 0.908],
                                    [0.14,  0.211, 0.262, 0.387],
                                    [0.215, 0.241, 0.257, 0.288]]])
print(np.round(propabilities, 3))
values = propabilities*values
np.round(np.amax(values.sum(2),1),3)
#values.sum(2)


[[[0.451 0.334 0.197 0.018]
  [0.271 0.27  0.258 0.2  ]
  [0.255 0.256 0.253 0.237]]

 [[0.14  0.307 0.366 0.187]
  [0.221 0.254 0.266 0.259]
  [0.24  0.251 0.255 0.254]]

 [[0.017 0.105 0.244 0.634]
  [0.177 0.234 0.268 0.322]
  [0.227 0.246 0.256 0.271]]

 [[0.001 0.017 0.075 0.908]
  [0.14  0.211 0.262 0.387]
  [0.215 0.241 0.257 0.288]]]


array([49. , 55.3, 87.8, 98.3])

In [92]:
portfolios = np.array([[0.0526, 0.0374], [0.07059443, 0.103057], [0.0886, 0.1954]])
W0 = 100
T = 10
#infusions = [0,25000,25000,25000,25000,25000,25000,25000,25000,25000,25000]
infusions = np.zeros(13)
empty_goal = np.array([[0,0]])
goals = np.array([empty_goal, empty_goal, empty_goal, empty_goal, empty_goal, np.array([[0,0],[60,200]]),empty_goal, empty_goal, empty_goal, empty_goal,np.array([[0,0],[120,1500]])])
iMax = 400

grid = generateGrid(W0,T,iMax, infusions, goals, portfolios[0,0], portfolios[0,1], portfolios[2,0], portfolios[2,1])
grid = grid.T

C:\Users\matsz\AppData\Local\Temp\ipykernel_20648\1125093231.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  goals = np.array([empty_goal, empty_goal, empty_goal, empty_goal, empty_goal, np.array([[0,0],[60,200]]),empty_goal, empty_goal, empty_goal, empty_goal,np.array([[0,0],[120,1500]])])


AttributeError: 'numpy.ndarray' object has no attribute 'get'

In [ ]:
W0 = 100
meanMax = 0.0886
stdMin = 0.0374
stdMax = 0.1954
t =11

W0*np.exp((meanMax-(stdMin**2/2))*t + 3*stdMax*np.sqrt(t))

1837.722768128645

In [ ]:
k = np.array([[105,100],[110,200]])
wealthInT = np.array([90,95,100,105,110])
Wk = np.tile(wealthInT, (2,1))
ki = np.tile(k, (2, 5,1))
#np.where(Wk >= k[:,0], k[:,1], 0)
ki

array([[[105, 100],
        [110, 200],
        [105, 100],
        [110, 200],
        [105, 100],
        [110, 200],
        [105, 100],
        [110, 200],
        [105, 100],
        [110, 200]],

       [[105, 100],
        [110, 200],
        [105, 100],
        [110, 200],
        [105, 100],
        [110, 200],
        [105, 100],
        [110, 200],
        [105, 100],
        [110, 200]]])

In [ ]:
def get_ef_port_analytically(rets, covm, targ):
    """Solve for the efficient frontier weights for a given expected return
    vector `rets`, covariance matrix `covm`, and expected portfolio return
    `targ`.
    """
    N = rets.shape[0]
    u = np.array([targ, 1])[:, None]
    U = np.vstack([rets, np.ones_like(rets)]).T

    covm_inv = np.linalg.solve(covm, np.eye(N))
    M        = U.T @ covm_inv @ U
    M_inv    = np.linalg.solve(M, np.eye(2))
    weights  = covm_inv @ U @ M_inv @ u

    return weights


In [ ]:
cov_tab = np.array([[0.0002, -0.00],
                    [-0.00, 0.0013]])
means = np.array([0.0307, 0.1094])

mu = 0.0744
std = 0.0712

w = get_ef_port_analytically(means,cov_tab,mu)

w

array([[0.44472681],
       [0.55527319]])

In [ ]:
def __prob(W0, W1, mean, std, h):
    return norm.pdf((np.log(W1/W0)-(mean-0.5*std**2)*h)/(std*np.sqrt(h)))
portfolios = np.array([[0.0526, 0.0374], [0.07059443, 0.103057  ], [0.0886, 0.1954]])
W0 = np.array([49,50,60,80,100,105])
W1 = np.array([90,95,100,103,105,110])
infusions = 5
h = 1
__prob(W0[1],W1[0],portfolios[0,0],portfolios[0,1],h)

1.0453784911703342e-45

In [ ]:
goals = np.array([[0,0,],[40,50], [60,100], [100,150]])
WT = np.array([39, 40, 59,60, 80, 100])
expected = np.array([0,50,50,100,100, 150])

result, goal_strategies = calculateValuesForLastPeriod(WT, goals)
print(result)
print(goal_strategies)

[  0.  50.  50. 100. 100. 150.]
[0 1 1 2 2 3]


In [ ]:
portfolios = np.array([[0.0526, 0.0374], [0.07059443, 0.103057  ], [0.0886, 0.1954]])
W0 = np.array([60, 80,100,105])
W1 = np.array([100,103,105,110])
infusions = 5

h = 1

values = [0,0,100,100]

propabilities = calculateTransitionPropabilitiesForAllPorfolios(portfolios, W0, W1, infusions, h)

print(np.round(propabilities, 3))


[[[1.    0.    0.    0.   ]
  [0.926 0.065 0.008 0.   ]
  [0.017 0.105 0.244 0.634]
  [0.001 0.017 0.075 0.908]]

 [[0.65  0.226 0.108 0.016]
  [0.388 0.284 0.222 0.106]
  [0.177 0.234 0.268 0.322]
  [0.14  0.211 0.262 0.387]]

 [[0.368 0.275 0.225 0.133]
  [0.287 0.264 0.247 0.202]
  [0.227 0.246 0.256 0.271]
  [0.215 0.241 0.257 0.288]]]


In [ ]:
np.round(propabilities * np.array([0,0,100,100]),2)


array([[[ 0.  ,  0.  ,  0.  ,  0.  ],
        [ 0.  ,  0.  ,  0.83,  0.  ],
        [ 0.  ,  0.  , 24.44, 63.36],
        [ 0.  ,  0.  ,  7.45, 90.76]],

       [[ 0.  ,  0.  , 10.84,  1.59],
        [ 0.  ,  0.  , 22.2 , 10.57],
        [ 0.  ,  0.  , 26.76, 32.16],
        [ 0.  ,  0.  , 26.24, 38.67]],

       [[ 0.  ,  0.  , 22.47, 13.25],
        [ 0.  ,  0.  , 24.7 , 20.2 ],
        [ 0.  ,  0.  , 25.6 , 27.09],
        [ 0.  ,  0.  , 25.68, 28.75]]])

In [ ]:
portfolios = np.array([[0.0526, 0.0374], [0.07059443, 0.103057  ], [0.0886, 0.1954]])
WT = np.array([49,50,60,80,100,105,180,190])
WT1 = np.array([90,95,100,103,105,110, 190,200])
VT1 = [00,00,00,00,100,100, 100, 100]
c = 50
#goals = [[50, 80], [30, 50],]
VTc0 = np.array([29.014, 29.55, 77.286, 95.997])

probabilities = calculateTransitionPropabilitiesForAllPorfolios(portfolios,WT,WT1,5,1)
print(np.round(probabilities,3))
""""
portfolios_strategies, VT, chosen_propabilities, V = get_portfolios_strategies(VT1,probabilities)
print(portfolios_strategies)
print(VT)
print(np.round(chosen_propabilities,3))
print(np.round(V,3)) 
"""
porfolios_strategies, VT, chosen_propabilities = calculateBelman(VT1,probabilities)
print(np.round(porfolios_strategies,1))
print(np.round(VT,1))
print(np.round(chosen_propabilities,2))


[[[1.    0.    0.    0.    0.    0.    0.    0.   ]
  [1.    0.    0.    0.    0.    0.    0.    0.   ]
  [1.    0.    0.    0.    0.    0.    0.    0.   ]
  [0.769 0.221 0.01  0.001 0.    0.    0.    0.   ]
  [0.    0.    0.017 0.105 0.244 0.633 0.    0.   ]
  [0.    0.    0.001 0.017 0.075 0.908 0.    0.   ]
  [0.    0.    0.    0.    0.    0.    0.504 0.496]
  [0.    0.    0.    0.    0.    0.    0.128 0.872]]

 [[0.91  0.082 0.006 0.001 0.    0.    0.    0.   ]
  [0.901 0.089 0.008 0.002 0.001 0.    0.    0.   ]
  [0.769 0.178 0.035 0.012 0.006 0.001 0.    0.   ]
  [0.281 0.255 0.18  0.132 0.103 0.049 0.    0.   ]
  [0.03  0.08  0.158 0.208 0.238 0.286 0.    0.   ]
  [0.016 0.053 0.131 0.196 0.244 0.36  0.    0.   ]
  [0.    0.    0.    0.    0.    0.    0.484 0.516]
  [0.    0.    0.    0.    0.    0.    0.421 0.579]]

 [[0.472 0.243 0.121 0.078 0.058 0.027 0.    0.   ]
  [0.461 0.244 0.124 0.081 0.061 0.029 0.    0.   ]
  [0.353 0.237 0.151 0.113 0.092 0.054 0.    0.   ]
  [0.196

In [ ]:
v = np.array([[100,100,100,100,100,100],
              [100,100,100,100,100,100],
              [100,100,100,100,100,100]])
np.argmax(np.round(VT,4),0)

array([0, 0, 0, 0, 0, 0], dtype=int64)

In [ ]:
n = 10.123456
np.round(n,4)

10.1235

In [177]:
def __calculateWtc(WT, goals_costs, infusion):        
    k = len(goals_costs)
    cf = goals_costs - infusion
    Wtc = np.tile(WT,(k,1)) - cf.reshape((k,1))
    return Wtc
goals = np.array([[40,10],[49, 15],[60, 30]])
WT = np.array([49,50,60,80,100,105])
infusion = 5
__calculateWtc(WT,goals[:,0],infusion)

array([[14, 15, 25, 45, 65, 70],
       [ 5,  6, 16, 36, 56, 61],
       [-6, -5,  5, 25, 45, 50]])

In [ ]:
def __calculate_drift(goal_porfolio_strategies, portfolios, h=1):
    portfolios_measures = np.take(portfolios, goal_porfolio_strategies, axis=0)
    b = (portfolios_measures[:,:,0]-0.5*portfolios_measures[:,:,1]**2)*h
    c = portfolios_measures[:,:,1]*np.sqrt(h)
    return portfolios_measures, b, c

goal_porfolio_strategies = np.array([[2,2,1,0],
                                    [2,2,2,1]])

portfolios = np.array([[0.0526, 0.0374], [0.07059443, 0.103057  ], [0.0886, 0.1954]])

__calculate_drift(goal_porfolio_strategies, portfolios)

#b result porfolio 2, 0.06950942




(array([[[0.0886    , 0.1954    ],
         [0.0886    , 0.1954    ],
         [0.07059443, 0.103057  ],
         [0.0526    , 0.0374    ]],
 
        [[0.0886    , 0.1954    ],
         [0.0886    , 0.1954    ],
         [0.0886    , 0.1954    ],
         [0.07059443, 0.103057  ]]]),
 array([[0.06950942, 0.06950942, 0.06528406, 0.05190062],
        [0.06950942, 0.06950942, 0.06950942, 0.06528406]]),
 array([[0.1954  , 0.1954  , 0.103057, 0.0374  ],
        [0.1954  , 0.1954  , 0.1954  , 0.103057]]))

In [153]:
from investment_planner import __get_porfolios_strategy_for_wealth_values, calculateTransitionPropabilitiesForGoals

goals = np.array([[5,5],[49, 15],[60, 30]])
WT = np.array([49,50,60,80,100,105])
#WT1 = np.array([90,95,100,103,105,110])
WT1 = np.array([90,95,100,105,110])
infusion = 5
porfolios = np.array([[0.0526, 0.0374], [0.07059443, 0.103057  ], [0.0886, 0.1954]])
''' Wtc = np.array([[44, 45, 55, 75, 95, 100],
                [ 5,  6, 16, 36, 56, 61],
                [-6, -5,  5, 25, 45, 50]]) '''

Wtc = np.array([[45, 55, 75, 95, 100],
                [ -16, 0, 36, 56, 61]])

portfolios_for_goals = np.array([[[0.0886, 0.1954],
                                  [0.0886, 0.1954],
                                  [0.0886, 0.1954],
                                  [0.0886, 0.1954],                                  
                                  [0.0526, 0.0374]],                        
                                 [[0.0886, 0.1954],
                                  [0.0886, 0.1954],                                  
                                  [0.0886, 0.1954],
                                  [0.0886, 0.1954],
                                  [0.0886, 0.1954]],                                  
                                  ])

prob = calculateTransitionPropabilitiesForGoals(Wtc, WT1, portfolios_for_goals,1)
print(np.round(prob, 3))




[[[0.612 0.244 0.095 0.036 0.014]
  [0.501 0.265 0.135 0.067 0.032]
  [0.308 0.253 0.195 0.143 0.101]
  [0.175 0.201 0.213 0.212 0.198]
  [0.    0.012 0.2   0.522 0.267]]

 [[  nan   nan   nan   nan   nan]
  [  nan   nan   nan   nan   nan]
  [0.712 0.207 0.059 0.017 0.005]
  [0.491 0.266 0.139 0.07  0.034]
  [0.438 0.268 0.157 0.088 0.048]]]


nan

In [130]:
probabilities = np.array([[[0.4, 0.4, 0.2, 0],
                           [0.4, 0.3, 0.2, 0.1],
                           [0.2, 0.3, 0.3, 0.2],
                           [0.1, 0.3, 0.4, 0.2]
                           ],
                           [[np.nan, np.nan, np.nan, np.nan],                            
                            [0.4, 0.4, 0.2, 0],
                            [0.1, 0.2, 0.4, 0.3],
                            [0.1, 0.2, 0.5, 0.2]
                           ],[
                           [np.nan, np.nan, np.nan, np.nan],  
                           [np.nan, np.nan, np.nan, np.nan],                         
                            [0.4, 0.4, 0.2, 0],
                            [0.4, 0.3, 0.2, 0.1]                            
                           ]])    

probabilities * [0,0,100,150]

array([[[ 0.,  0., 20.,  0.],
        [ 0.,  0., 20., 15.],
        [ 0.,  0., 30., 30.],
        [ 0.,  0., 40., 30.]],

       [[nan, nan, nan, nan],
        [ 0.,  0., 20.,  0.],
        [ 0.,  0., 40., 45.],
        [ 0.,  0., 50., 30.]],

       [[nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [ 0.,  0., 20.,  0.],
        [ 0.,  0., 20., 15.]]])

In [ ]:
def __prob(W0, W1, mean, std, Infusion, Cost, h):
    return norm.pdf((np.log(W1/(W0+Infusion+Cost))-(mean-0.5*std**2)*h)/(std*np.sqrt(h)))

print((0.0526-0.5*0.0374**2))
print(__prob(100,90,0.0526, 0.0374, 0,0,1))
np.log(105/95)
#np.round(np.expand_dims(prob.sum(1), 1),3)
norm.pdf((np.log(110/100)-0.05190062)/0.0374)

0.05190062
5.77555159403567e-05


0.20341002067644562

In [114]:
def __get_porfolio_strategy_for_wealth_values(WT, wealth_value, porfolios_strategies):
    k = wealth_value.shape[0]
    i = wealth_value.shape[1]
    wc = wealth_value.reshape(k*i,1)
    wg = np.tile(WT,(k*i,1))
    difference = np.absolute(wg - wc)
    index = np.argmin(difference, axis=1)
    index = index.reshape(k,i)
    return np.take(porfolios_strategies,index)

porfolios_strategies = [2,2,2,1,0,0]
WT = np.array([49,50,60,80,100,105])
goals = np.array([[5,5],[20,10],[60, 30]])
Wtc = np.tile(WT,(len(goals),1)) - goals[:,0].reshape(len(goals),1)
#result [[2,2,2,1,0,0],
#        [2,2,2,2,1,1],
#        [2,2,2,2,2,2]
print(WT)
print(Wtc)

__get_porfolio_strategy_for_wealth_values(WT, Wtc, porfolios_strategies)


[ 49  50  60  80 100 105]
[[ 44  45  55  75  95 100]
 [ 29  30  40  60  80  85]
 [-11 -10   0  20  40  45]]


array([[2, 2, 2, 1, 0, 0],
       [2, 2, 2, 2, 1, 1],
       [2, 2, 2, 2, 2, 2]])

In [7]:

from model.investment_planner import __get_porfolios_strategy_for_wealth_values, calculateTransitionPropabilitiesForGoals, calculateBelmanForT

%precision 3

portfolios = np.array([[0.0526, 0.0374], [0.07059443, 0.103057  ], [0.0886, 0.1954]])
WT = np.array([34,50,60,80,100,105,120])
WT1 = np.array([36,95,100,103,105,110,130])
#WT = np.linspace(50,100,20)
#WT1 = np.linspace(45,120,20)
#VT1 = [0,0,0,100,100,100,100]
VT1 = [0,0,0,00,100,100,100]
#VT1 = np.zeros(20)
#VT1[12:-1] = 100
goals = np.array([[40,110],[49, 120],[80, 130]])
#goals = np.array([[20,150]])
#goals = np.array([[0,0]])
infusion = 5

Wt = WT

goal_strategies, portfolio_strategies, values, probabilities = calculateBelmanForT(goals, infusion, WT,WT1, VT1, portfolios)

print(goal_strategies)
print(portfolio_strategies)
print(np.round(values,1))
print(np.round(probabilities,2))
probabilities.shape


[0 2 2 3 2 2 2]
[2. 2. 2. 2. 2. 2.]
[  0.  120.  120.  130.  134.3 140.2 150.7]
[[1.   0.   0.   0.   0.   0.   0.  ]
 [1.   0.   0.   0.   0.   0.   0.  ]
 [1.   0.   0.   0.   0.   0.   0.  ]
 [1.   0.   0.   0.   0.   0.   0.  ]
 [0.18 0.36 0.19 0.12 0.09 0.05 0.  ]
 [0.02 0.39 0.23 0.16 0.13 0.07 0.  ]
 [0.   0.28 0.22 0.19 0.17 0.12 0.02]]


(7, 7)

In [22]:
100*np.exp(portfolios[2,0]-3*portfolios[2,1]*np.sqrt(3))
np.exp(np.linspace(np.log(40), np.log(300),6)) + 11

array([ 51.   ,  70.851, 100.554, 144.998, 211.498, 311.   ])

In [197]:
Wtc = __calculateWtc(WT, goals[:,0], infusion)
goal_portfolio_strategies = __get_porfolios_strategy_for_wealth_values(WT,Wtc, [2,2,2,2,2,2,2])
portfolios_measures = np.take(portfolios, goal_portfolio_strategies, axis=0)
np.round(calculateTransitionPropabilitiesForGoals(Wtc,WT1,portfolios_measures),3)
goal_portfolio_strategies


array([[2, 2, 2, 2, 2, 2, 2],
       [2, 2, 2, 2, 2, 2, 2],
       [2, 2, 2, 2, 2, 2, 2]])

In [221]:
goal_portfolios_strategies = np.array([[2,2,1,1,0,0],
                                      [2,2,2,2,1,1],
                                      [2,2,2,2,2,1]])
goal_strategies = np.array([0,0,1,1,2,2])

chosen_strategies = np.zeros(6)
for i in range(0, goal_portfolios_strategies.shape[1]):
    chosen_strategies[i] = goal_portfolios_strategies[goal_strategies[i],i]

chosen_strategies

array([2., 2., 2., 2., 2., 1.])

In [196]:
def __prob(W0, W1, mean, std, Infusion, Cost, h):
    return norm.pdf((np.log(W1/(W0+Infusion+Cost))-(mean-0.5*std**2)*h)/(std*np.sqrt(h)))

np.round(__prob(Wtc[1,3],WT1,porfolios[2,0], portfolios[2,1],0,0,1),3)

array([0.374, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ])

In [162]:
goals = np.array([[0,0]])

goal_strategies, portfolio_strategies, values, probabilities = calculateBelman(goals, infusion, WT,WT1, VT1, portfolios)

print(goal_strategies)
print(portfolio_strategies)
print(np.round(values,1))
print(np.round(probabilities,2))  

[0 0 0 0 0 0 0]
[[2 2 2 2 0 0 0]]
[  0.   12.4  23.2  37.1  87.8  98.2 100. ]
[[1.   0.   0.   0.   0.   0.   0.  ]
 [0.27 0.33 0.17 0.11 0.08 0.04 0.  ]
 [0.   0.36 0.23 0.17 0.14 0.08 0.01]
 [0.   0.23 0.21 0.19 0.18 0.15 0.04]
 [0.   0.   0.02 0.1  0.24 0.63 0.  ]
 [0.   0.   0.   0.02 0.07 0.89 0.02]
 [0.   0.   0.   0.   0.   0.   1.  ]]


```
[[16.4 17.2 25.9 43.6 98.3 99.9]
 [10.2 10.2 11.4 19.8 35.9 40.5]
 [15.  15.  15.3 19.9 33.  37.3]
 [80.  80.  30.  31.4 39.8 43.3]]
```

In [ ]:
def __prob(W0, W1, mean, std, Infusion, Cost, h):
    return norm.pdf((np.log(W1/(W0+Infusion+Cost))-(mean-0.5*std**2)*h)/(std*np.sqrt(h)))

__prob(105,90,porfolios[2,0],porfolios[2,1],5,0,1)

0.153

```
expected values array([24.794, 29.794, 29.794, 44.794, 95.456, 99.669])
array([[14.794, 15.581, 24.116, 41.942, 95.456, 99.669],
       [24.794, 24.794, 24.794, 24.794, 34.116, 34.116],
       [14.794, 29.794, 29.794, 29.794, 30.581, 39.116],
       [14.794, 15.581, 24.116, 44.794, 44.794, 44.794]])

```

In [166]:
from investment_planner import InvestmentPlanner 

investment_planner = InvestmentPlanner()

empty_goal = np.array([[0,0]])
goals = np.array([empty_goal, empty_goal, empty_goal, empty_goal, empty_goal, np.array([[0,0],[100,50]]),empty_goal, empty_goal, empty_goal, empty_goal,np.array([[0,0],[120,1500]])])

#portfolios = np.array([[0.0526, 0.0374], [0.07059443, 0.103057  ], [0.0886, 0.1954]])
portfolios = np.array([[0.05258386, 0.03704926],
       [0.05515672, 0.03960988],
       [0.05772681, 0.04625568],
       [0.06029967, 0.05555016],
       [0.06286483, 0.06637403],
       [0.06545148, 0.07813246],
       [0.06801664, 0.09041086],
       [0.07059443, 0.103057  ],
       [0.07315959, 0.11592848],
       [0.07573245, 0.12898073],
       [0.07830254, 0.14213997],
       [0.0808754 , 0.15540374],
       [0.08344549, 0.16872224],
       [0.08601835, 0.18210899],
       [0.08858351, 0.19552512]])

investment_planner.set_params(10, 100, 10, 1, goals, portfolios)

glide_path = investment_planner.glide_paths
wealth_grid = investment_planner.grid
probabilitiesT = investment_planner.probabilitiesT
goal_strategies = investment_planner._goal_strategies
#probabilities = investment_planner.propabilities

#TODO do grid trzeba dodac goals cost max, ael trzeba sprawdzic

Unexpected exception formatting exception. Falling back to standard exception


C:\Users\matsz\AppData\Local\Temp\ipykernel_20648\4245196626.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  goals = np.array([empty_goal, empty_goal, empty_goal, empty_goal, empty_goal, np.array([[0,0],[100,50]]),empty_goal, empty_goal, empty_goal, empty_goal,np.array([[0,0],[120,1500]])])
Traceback (most recent call last):
  File "c:\Users\matsz\miniconda3\envs\portfolio-optimisation\lib\site-packages\IPython\core\interactiveshell.py", line 3397, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\matsz\AppData\Local\Temp\ipykernel_20648\4245196626.py", line 25, in <cell line: 25>
    investment_planner.set_params(10, 100, 10, 1, goals, portfolios)
  File "c:\Users\matsz\programowanie\Optymalizacja portfela\investment_planner.py", 

In [229]:
from investment_planner import InvestmentPlanner 

investment_planner = InvestmentPlanner()

empty_goal = np.array([[0,0]])
#goals = np.array([empty_goal,empty_goal,np.array([[0,0],[110,100]])])
goals = [{
        "time": 5,
        "cost": 140,
        "utility": 500                
    }]

#portfolios = np.array([[0.0526, 0.0374], [0.07059443, 0.103057  ], [0.0886, 0.1954]])
portfolios = np.array([[0.05258386, 0.03704926],
       [0.05515672, 0.03960988],
       [0.05772681, 0.04625568],
       [0.06029967, 0.05555016],
       [0.06286483, 0.06637403],
       [0.06545148, 0.07813246],
       [0.06801664, 0.09041086],
       [0.07059443, 0.103057  ],
       [0.07315959, 0.11592848],
       [0.07573245, 0.12898073],
       [0.07830254, 0.14213997],
       [0.0808754 , 0.15540374],
       [0.08344549, 0.16872224],
       [0.08601835, 0.18210899],
       [0.08858351, 0.19552512]])

investment_planner.set_params(5, 100, 0, 1, goals, portfolios)

glide_path = investment_planner.glide_paths
wealth_grid = investment_planner.grid.T
probabilitiesT = np.round(investment_planner.probabilitiesT,3)
goal_strategies = investment_planner._goal_strategies
v = investment_planner.V
cum_probs = np.round(investment_planner.cum_propabilities,3)
#probabilities = investment_planner.propabilities

#TODO naprawić goal strategies


In [ ]:
goal_propabilities = investment_planner.get_goal_propabilities()


In [ ]:
def __prob(W0, W1, mean, std, Infusion, Cost, h):
    return norm.pdf((np.log(W1/(W0+Infusion+Cost))-(mean-0.5*std**2)*h)/(std*np.sqrt(h)))


WT1 = wealth_grid[:,5]
WT = wealth_grid[87,4]
VT = goal_strategies*100
for porfolio in portfolios:
    #print(np.round(np.sum(__prob(WT,WT1,porfolio[0], porfolio[1],0,0,1)*VT),3))
    #print(np.round(__prob(WT,WT1,porfolio[0], porfolio[1],0,0,1)*VT,1))
    #print(np.round(__prob(WT,WT1,porfolio[0], porfolio[1],0,0,1),1))
    prob = __prob(WT,WT1,porfolio[0], porfolio[1],0,0,1)
    prob = prob / prob.sum()
    print(np.round(np.sum(prob*VT), 4))
    


143.2293
147.6829
153.9683
160.2346
165.6733
170.1174
173.5549
176.0634
177.7522
178.8007
179.3944
179.6943
179.8149
179.8313
179.7878


In [ ]:
probs = calculateTransitionPropabilitiesForAllPorfolios(portfolios,wealth_grid[4],WT1,0)
strategies, V, chosen_probs = calculateBelman(VT,probs)
print(strategies)
print(V)
print(np.round(chosen_probs, 3))

[0 2 2 2 2 0]
[71.21347431 30.30395294 24.83908031 21.92699645 20.29551439         nan]
[[0.    0.008 0.139 0.305 0.363 0.186]
 [0.403 0.242 0.139 0.098 0.077 0.041]
 [0.522 0.24  0.107 0.065 0.047 0.02 ]
 [0.605 0.226 0.083 0.045 0.03  0.011]
 [0.664 0.211 0.066 0.033 0.021 0.006]
 [  nan   nan   nan   nan   nan   nan]]


c:\Users\matsz\programowanie\Optymalizacja portfela\investment_planner.py:108: RuntimeWarning: invalid value encountered in true_divide
  return np.divide(result, np.expand_dims(result.sum(2), axis=2), where=result.sum()>0)


In [ ]:
def WMax(t,W0, infusions, meanMax,stdMin,stdMax):
    valueOfInfusions = 0
    for i in range(t):
        valueOfInfusions += infusions[i]*np.exp((meanMax - (stdMin**2)/2)*(t-i) + (3*stdMax*np.sqrt(t-i)))
                                                           
    return W0*np.exp((meanMax-((stdMin**2)/2))*t + 3*stdMax*np.sqrt(t)) + valueOfInfusions

WMax(2,100,[0,0,0,0,0,0,0,0,0,0,0,0],portfolios[14,0], portfolios[0,1], portfolios[14,1])

273.283

In [ ]:
arr = np.array([[[0,0]],[[0,0]],[[0,0]],[[0,0]],[[0,0]],np.array([[0,0],[120,1000], [220,1000]]),[[0,0]],[[0,0]],[[0,0]],[[0,0]],np.array([[0,0],[140,1500]])])
T = 10
for t in range(1,T+1):
    cmax = arr[t][:,0].max() if (len(arr[t]) >1) else 0
    print(cmax)  



0
0
0
0
220
0
0
0
0
140


C:\Users\matsz\AppData\Local\Temp\ipykernel_41080\3029623270.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.array([[[0,0]],[[0,0]],[[0,0]],[[0,0]],[[0,0]],np.array([[0,0],[120,1000], [220,1000]]),[[0,0]],[[0,0]],[[0,0]],[[0,0]],np.array([[0,0],[140,1500]])])


In [172]:
def calculateTransitionPropabilitiesForGoals(Wt, Wt1, infusion, h, goal_costs, portfolioMeasures):
    i0 = len(Wt)
    i1 = len(Wt1)    
    k = len(goal_costs)
    cf = goal_costs - infusion
    mean = portfolioMeasures[0]
    std = portfolioMeasures[1]
    b = ((mean-0.5*std**2)*h)/(std*np.sqrt(h))
    result = np.zeros((k*i0,i1))
    
    Wtc = np.tile(Wt,(k,1)) - cf.reshape((k,1))
    Wtc = Wtc.reshape((k*i0,1))
    Wt1k = np.tile(Wt1, (k*i0,1))
    
    np.divide(Wt1k, Wtc, out=result, where=Wtc>0)
    np.log(result,out=result, where=result>0)
    result = np.where(result > 0, result+ b, 0)
    result = np.where(result > 0, norm.pdf(result), 0)

    result = np.divide(result,np.expand_dims(result.sum(1), axis=1),where=result>0)
    result = result.reshape(k,i0,i1)

    return result
   

portfolios = np.array([[0.0526, 0.0374], [0.07059443, 0.103057  ], [0.0886, 0.1954]])
WT = np.array([49,50,60,80,100,105])
#WT = np.array([100])
WT1 = [90,95,100,103,105,110]

goals = np.array([[40,10],[49, 15],[60, 30]])
#goals = np.array([[49, 15],[60, 30]])
#goals = np.array([[60,0]])
infusion = 5
h = 1

probabilities = calculateTransitionPropabilitiesForGoals(WT, WT1, infusion, h, goals[:,0], [0.1,0.2])

print(np.round(probabilities,3))
probabilities.sum(1)



[[[0.212 0.187 0.166 0.155 0.148 0.132]
  [0.211 0.187 0.166 0.155 0.148 0.133]
  [0.2   0.182 0.167 0.158 0.153 0.14 ]
  [0.189 0.177 0.167 0.161 0.157 0.148]
  [0.181 0.174 0.167 0.163 0.16  0.154]
  [0.18  0.174 0.167 0.163 0.161 0.155]]

 [[0.234 0.195 0.164 0.149 0.139 0.118]
  [0.23  0.194 0.165 0.15  0.141 0.121]
  [0.209 0.186 0.166 0.156 0.149 0.134]
  [0.193 0.179 0.167 0.16  0.156 0.145]
  [0.184 0.176 0.167 0.162 0.159 0.152]
  [0.183 0.175 0.167 0.163 0.16  0.153]]

 [[0.    0.    0.    0.    0.    0.   ]
  [0.    0.    0.    0.    0.    0.   ]
  [0.234 0.195 0.164 0.149 0.139 0.118]
  [0.2   0.182 0.167 0.158 0.153 0.14 ]
  [0.189 0.177 0.167 0.161 0.157 0.148]
  [0.186 0.177 0.167 0.162 0.158 0.15 ]]]


array([[1.173, 1.082, 1.   , 0.956, 0.928, 0.862],
       [1.233, 1.105, 0.997, 0.939, 0.903, 0.823],
       [0.809, 0.732, 0.665, 0.63 , 0.607, 0.557]])

```
[[0.    0.    0.    0.    0.    0.   ]
 [0.001 0.001 0.001 0.001 0.    0.   ]
 [0.016 0.014 0.012 0.012 0.011 0.01 ]
 [0.091 0.083 0.076 0.072 0.069 0.064]
 [0.177 0.165 0.154 0.148 0.144 0.134]
 [0.197 0.184 0.173 0.166 0.162 0.152]
 [0.    0.    0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.    0.   ]
 [0.046 0.041 0.037 0.034 0.033 0.03 ]
 [0.131 0.121 0.111 0.106 0.103 0.095]
 [0.152 0.141 0.131 0.125 0.122 0.113]]
 ```

In [ ]:
arr =  np.array([0.197, 0.184, 0.173, 0.166, 0.162, 0.152])
res = arr/ arr.sum()
np.expand_dims(res, axis=1)


array([[0.191],
       [0.178],
       [0.167],
       [0.161],
       [0.157],
       [0.147]])

In [ ]:
log = [1.38778666, 1.44185389, 1.49314718, 1.52270598, 1.54193734, 1.58845736]
log2 =  [3.69037176, 3.74443898, 3.79573227, 3.82529108, 3.84452244, 3.89104245]
norm.pdf(log2)

array([0., 0., 0., 0., 0., 0.])

1: 48.057
2. 64.226
m. 56.305

In [ ]:
stocks = norm.rvs(loc=0.09,scale=0.19,size=30, random_state=13)
bonds = norm.rvs(loc=0.04,scale=0.03, size=30, random_state=13)
returns = np.array([stocks,bonds]).T
weights = np.array([0.5,0.5])
porfolio_returns = returns @ weights
var = weights.T @ np.cov(returns.T) @ weights
cov = np.cov(stocks, porfolio_returns)[0,1]
beta = cov/var
porfolio_returns.mean()*beta





0.146

In [ ]:
def _calculate_cumulative_propabilities(probabilitiesT):
        i = probabilitiesT[0].shape[1]
        T = probabilitiesT.shape[0]
        cumulativeProbabilities = np.zeros((T, probabilitiesT[0].shape[1]))
        cumulativeProbabilities[0,:] = 1
        cumulativeProbabilities[1] = probabilitiesT[0,0]
        for t in range(2, T):
                for it in range(0,i):                      
                        cumulativeProbabilities[t,it] = np.sum(probabilitiesT[t-1,:,it]*cumulativeProbabilities[t-1,it])
        return cumulativeProbabilities

prob = np.array([[[0.1,0.3,0.3,0.2,0.1],
                  [0.1,0.3,0.3,0.2,0.1],
                  [0.1,0.3,0.3,0.2,0.1],
                  [0.1,0.3,0.3,0.2,0.1],
                  [0.1,0.3,0.3,0.2,0.1]
                  ],
                  [[0,0.4,0.3,0.2,0.1],
                  [0,0.3,0.4,0.2,0.1],
                  [0,0.2,0.4,0.3,0.1],
                  [0,0.1,0.4,0.4,0.1],
                  [0,0.1,0.3,0.3,0.3]
                  ],
                  [[0,0.4,0.3,0.2,0.1],
                  [0,0.3,0.4,0.2,0.1],
                  [0,0.2,0.4,0.3,0.1],
                  [0,0.1,0.4,0.4,0.1],
                  [0,0.1,0.3,0.3,0.3]
                  ],                  
                  [[0,0.4,0.3,0.2,0.1],
                  [0,0.3,0.4,0.2,0.1],
                  [0,0.2,0.4,0.3,0.1],
                  [0,0.1,0.4,0.4,0.1],
                  [0,0.1,0.3,0.3,0.3]
                  ]])

_calculate_cumulative_propabilities(prob)
#expected[1]
#expected = [0.5,0.9, 1.8,1.2,0.6]
#prob.sum(1)

array([[1.   , 1.   , 1.   , 1.   , 1.   ],
       [0.1  , 0.3  , 0.3  , 0.2  , 0.1  ],
       [0.   , 0.33 , 0.54 , 0.28 , 0.07 ],
       [0.   , 0.363, 0.972, 0.392, 0.049]])